Cell 1 - Imports and Config

In [17]:
import os
import sys
import math
import time
import wave
import struct
from io import BytesIO
from typing import List, Dict, Any, Tuple

import numpy as np
from PIL import Image, ImageDraw, ImageFont

# Widgets for uploads & controls
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- MoviePy detection -------------------------------------------------------
HAS_MOVIEPY = False
HAS_LIBROSA = False
HAS_SVG = False

try:
    from moviepy.editor import VideoClip, AudioFileClip
    import moviepy
    HAS_MOVIEPY = True
    print(f"[Init] MoviePy detected: {moviepy.__version__}")
except Exception as e:
    print(f"[Warning] MoviePy unavailable: {e}")

try:
    import librosa
    HAS_LIBROSA = True
    print("[Init] Librosa detected.")
except Exception as e:
    print(f"[Warning] Librosa unavailable: {e}")

try:
    import cairosvg
    HAS_SVG = True
    print("[Init] CairoSVG detected.")
except Exception as e:
    print(f"[Warning] CairoSVG unavailable: {e}")

# ---------------------------------------------------------------------------
# Config
# ---------------------------------------------------------------------------

# IMPORTANT: we symlinked geometry → ~/projects/LightCraft/geometry
#   cd /var/www/schizo-studios/notebooks
#   ln -s ~/projects/LightCraft/geometry geometry
GEOMETRY_DIR = "geometry"

SACRED_NUMBER = 7      # numerology cycle (1–7)
ASTRO_CYCLES = 12      # zodiac cycle
RENDER_FPS = 30
MOCK_DURATION_S = 30.0

ZODIAC_ARCHETYPES = [
    {"sign": "Aries",       "element": "Fire",  "color": (200,  50,  50)},
    {"sign": "Taurus",      "element": "Earth", "color": ( 50, 150,  50)},
    {"sign": "Gemini",      "element": "Air",   "color": (150, 150, 200)},
    {"sign": "Cancer",      "element": "Water", "color": (100, 150, 200)},
    {"sign": "Leo",         "element": "Fire",  "color": (255, 160,   0)},
    {"sign": "Virgo",       "element": "Earth", "color": (100, 100, 100)},
    {"sign": "Libra",       "element": "Air",   "color": (200, 100, 200)},
    {"sign": "Scorpio",     "element": "Water", "color": ( 80,   0, 120)},
    {"sign": "Sagittarius", "element": "Fire",  "color": (180,  80,   0)},
    {"sign": "Capricorn",   "element": "Earth", "color": ( 50,  50,  80)},
    {"sign": "Aquarius",    "element": "Air",   "color": (  0, 150, 200)},
    {"sign": "Pisces",      "element": "Water", "color": (100, 100, 255)},
]

RESOLUTIONS = {
    1: (1920, 1080, "16:9 (HD/YouTube)"),
    2: (3840, 2160, "16:9 (4K UHD)"),
    3: (1080, 1920, "9:16 (Vertical/Shorts)"),
    4: (1080, 1080, "1:1 (Square/Instagram)"),
}


[Init] MoviePy detected: 1.0.3
[Init] Librosa detected.
[Init] CairoSVG detected.


Cell 2 - Helpers + Geometry Loader

In [14]:
from IPython.display import HTML
HTML('''
<style>
div.input {display:none;}
</style>
''')

def _interpolate_color(c1: Tuple[int,int,int], c2: Tuple[int,int,int], blend: float) -> Tuple[int,int,int]:
    blend = max(0.0, min(1.0, blend))
    return (
        int(c1[0] + (c2[0] - c1[0]) * blend),
        int(c1[1] + (c2[1] - c1[1]) * blend),
        int(c1[2] + (c2[2] - c1[2]) * blend),
    )

def _ensure_geometry_dir(geometry_dir: str) -> List[str]:
    if not HAS_SVG:
        raise RuntimeError("You must install `cairosvg` and `Pillow` for SVG support (pip install cairosvg pillow).")

    if not os.path.isdir(geometry_dir):
        raise RuntimeError(f"Geometry directory not found: {geometry_dir}")

    files = [
        os.path.join(geometry_dir, f)
        for f in sorted(os.listdir(geometry_dir))
        if f.lower().endswith(".svg")
    ]
    if not files:
        raise RuntimeError(f"No .svg files found in {geometry_dir}")

    print(f"[Init] Loaded {len(files)} sacred geometry SVGs from {geometry_dir}")
    return files


Cell 3 - SacredAnalyzer

In [10]:
class SacredAnalyzer:
    """Audio → sacred visual parameters using Librosa features."""

    def __init__(self, audio_data: np.ndarray, sample_rate: int, duration_s: float):
        self.audio_data = audio_data
        self.sample_rate = sample_rate
        self.duration_s = duration_s

        self.beat_times = np.array([])
        self.tonnetz = np.array([])
        self.frame_times = np.array([])
        self.phrase_boundaries = np.array([])
        self._current_cycle = 0

        if HAS_LIBROSA:
            self._pre_analyze()
        else:
            print("[Analyzer] Librosa not installed; running in minimal mode.")

    def _pre_analyze(self):
        if self.audio_data.size == 0:
            print("[Analyzer] No audio data to analyze.")
            return

        # Beat tracking
        _, self.beat_times = librosa.beat.beat_track(
            y=self.audio_data, sr=self.sample_rate, units="time"
        )
        print(f"[Analyzer] Beats detected: {len(self.beat_times)}")

        # Tonnetz (tonal centroid)
        chroma = librosa.feature.chroma_stft(y=self.audio_data, sr=self.sample_rate)
        self.tonnetz = librosa.feature.tonnetz(chroma=chroma)
        self.frame_times = librosa.frames_to_time(
            np.arange(self.tonnetz.shape[1]), sr=self.sample_rate
        )

        # Phrase boundaries via RMS
        rms = librosa.feature.rms(y=self.audio_data)[0]
        median_rms = np.median(rms)
        frames = librosa.util.peak_pick(
            rms, pre_max=20, post_max=20,
            pre_avg=10, post_avg=10,
            delta=median_rms * 0.5,
            wait=10
        )
        self.phrase_boundaries = librosa.frames_to_time(frames, sr=self.sample_rate)
        print(f"[Analyzer] Phrase boundaries: {len(self.phrase_boundaries)}")

    def get_params(self, t: float) -> Dict[str, Any]:
        if self.phrase_boundaries.size > 0:
            for b in self.phrase_boundaries:
                if b > (t - 1.0/RENDER_FPS) and b <= t:
                    self._current_cycle = (self._current_cycle % SACRED_NUMBER) + 1
                    break
        layers = self._current_cycle if self._current_cycle > 0 else 1

        # Tonnetz → consonance/dissonance
        if HAS_LIBROSA and self.tonnetz.size > 0:
            idx = np.searchsorted(self.frame_times, t, side="left")
            idx = min(idx, self.tonnetz.shape[1] - 1)
            curr = self.tonnetz[:, idx]
            if idx > 0:
                prev = self.tonnetz[:, idx - 1]
                change = np.linalg.norm(curr - prev)
            else:
                change = 0.0
            max_change = 0.8
            dissonance = min(1.0, change / max_change)
            consonance = 1.0 - dissonance
        else:
            consonance = 1.0
            dissonance = 0.0

        # Local amplitude
        start = int(t * self.sample_rate)
        end = min(start + self.sample_rate // RENDER_FPS, len(self.audio_data))
        if end > start:
            chunk = self.audio_data[start:end]
            peak = float(np.max(np.abs(chunk))) if chunk.size > 0 else 0.0
        else:
            peak = 0.0
        geom_intensity = min(1.0, peak * 5.0)

        # Beat pulse
        pulse = 0.0
        if self.beat_times.size > 0:
            idx = np.argmin(np.abs(self.beat_times - t))
            dt = abs(self.beat_times[idx] - t)
            tol = 0.1
            if dt < tol:
                pulse = 1.0 - (dt / tol)

        time_ratio = t / self.duration_s if self.duration_s > 0 else 0.0
        astro_phase = (time_ratio * ASTRO_CYCLES) % ASTRO_CYCLES

        return {
            "time_s": t,
            "layers": layers,
            "geom_intensity": geom_intensity,
            "consonance": consonance,
            "dissonance": dissonance,
            "pulse": pulse,
            "astro_phase": astro_phase,
        }


Cell 4 - SVGVisualGenerator

In [11]:
class SVGVisualGenerator:
    """
    Render sacred geometry using your SVGs with:
    - Zodiac-based background
    - Multi-tone radial gradient
    - SVG overlay (rotating, breathing)
    - Dual breathing: geometry + camera
    """

    def __init__(self, width: int, height: int, svg_paths: List[str]):
        self.width = width
        self.height = height
        self.svg_paths = svg_paths
        self.cache: Dict[str, Image.Image] = {}

    def _load_svg_rgba(self, path: str) -> Image.Image:
        if path in self.cache:
            return self.cache[path]

        with open(path, "rb") as f:
            svg_bytes = f.read()

        png_bytes = cairosvg.svg2png(
            bytestring=svg_bytes,
            output_width=self.width,
            output_height=self.height,
        )
        img = Image.open(BytesIO(png_bytes)).convert("RGBA")
        self.cache[path] = img
        return img

    def _zodiac_bg_color(self, astro_phase: float) -> Tuple[int,int,int]:
        idx1 = int(astro_phase) % ASTRO_CYCLES
        idx2 = (idx1 + 1) % ASTRO_CYCLES
        blend = astro_phase - idx1
        c1 = ZODIAC_ARCHETYPES[idx1]["color"]
        c2 = ZODIAC_ARCHETYPES[idx2]["color"]
        return _interpolate_color(c1, c2, blend)

    def _gradient_colors(self, params: Dict[str, Any]) -> Tuple[Tuple[int,int,int], Tuple[int,int,int]]:
        astro_phase = params["astro_phase"]
        layers = params["layers"]
        intensity = params["geom_intensity"]
        pulse = params["pulse"]

        base_phase = (astro_phase / ASTRO_CYCLES + layers / (SACRED_NUMBER * 2.0)) % 1.0

        def phase_to_rgb(p: float) -> Tuple[int,int,int]:
            r = 0.5 + 0.5 * math.cos(2*math.pi*p)
            g = 0.5 + 0.5 * math.cos(2*math.pi*p + 2.09)
            b = 0.5 + 0.5 * math.cos(2*math.pi*p + 4.18)
            return (int(r*255), int(g*255), int(b*255))

        inner = phase_to_rgb(base_phase)
        outer_phase = (base_phase + 0.15 + 0.3*pulse) % 1.0
        outer = phase_to_rgb(outer_phase)

        def scale_color(c, s):
            return tuple(min(255, int(ch*s)) for ch in c)

        bloom = 1.0 + 0.5 * intensity
        inner = scale_color(inner, 0.9 + 0.3*bloom)
        outer = scale_color(outer, 0.7 + 0.4*bloom)

        return inner, outer

    def _pick_svg_for_params(self, params: Dict[str, Any]) -> str:
        n = len(self.svg_paths)
        if n == 0:
            raise RuntimeError("No SVGs available.")

        layers = max(1, min(SACRED_NUMBER, params["layers"]))
        astro_phase = params["astro_phase"]

        base_idx = (layers - 1) % n
        offset = int(astro_phase) % n
        idx = (base_idx + offset) % n
        return self.svg_paths[idx]

    def make_frame(self, t: float, params: Dict[str, Any]) -> np.ndarray:
        bg_color = self._zodiac_bg_color(params["astro_phase"])
        bg = np.full((self.height, self.width, 3), bg_color, dtype=np.uint8)

        svg_path = self._pick_svg_for_params(params)
        base_img = self._load_svg_rgba(svg_path)

        dissonance = params["dissonance"]
        geom_intensity = params["geom_intensity"]
        pulse = params["pulse"]

        base_scale = 0.9 + 0.15*geom_intensity + 0.1*pulse
        rotation_deg = (t * 10.0 * (0.5 + dissonance)) % 360.0

        # Hypnotic breathing (4–4–6–2)
        cycle_time = 16.0
        phase = t % cycle_time

        if phase < 4.0:         # Inhale
            breath = 1.0 + (phase / 4.0)
            micro = 0.0
        elif phase < 8.0:       # Hold
            breath = 2.0
            micro = 0.02 * math.sin(2 * math.pi * (phase - 4.0) * 1.2)
        elif phase < 14.0:      # Exhale
            breath = 2.0 - ((phase - 8.0) / 6.0)
            micro = 0.0
        else:                   # Pause
            breath = 1.0
            micro = 0.03 * math.sin(2 * math.pi * (phase - 14.0) * 1.5)

        geometry_scale = breath + micro
        camera_scale = breath + micro * 0.5

        scale_factor = base_scale * geometry_scale

        w_base, h_base = base_img.size
        w_scaled = max(1, int(w_base * scale_factor))
        h_scaled = max(1, int(h_base * scale_factor))
        img_scaled = base_img.resize((w_scaled, h_scaled), resample=Image.LANCZOS)
        img_rot = img_scaled.rotate(rotation_deg, resample=Image.BICUBIC, expand=True)

        canvas = Image.new("RGBA", (self.width, self.height), (0,0,0,0))
        x = (self.width  - img_rot.size[0]) // 2
        y = (self.height - img_rot.size[1]) // 2
        canvas.paste(img_rot, (x, y), img_rot)

        svg_rgba = np.array(canvas).astype(np.float32)

        inner_color, outer_color = self._gradient_colors(params)

        yy, xx = np.meshgrid(
            np.linspace(-1.0, 1.0, self.height),
            np.linspace(-1.0, 1.0, self.width),
            indexing="ij",
        )
        radius = np.sqrt(xx**2 + yy**2)
        radius = np.clip(radius, 0.0, 1.0)

        inner = np.array(inner_color, dtype=np.float32).reshape(1,1,3)
        outer = np.array(outer_color, dtype=np.float32).reshape(1,1,3)
        grad_rgb = inner*(1.0 - radius[...,None]) + outer*radius[...,None]

        alpha = svg_rgba[..., 3:4] / 255.0
        geom_rgb = grad_rgb * alpha

        bg_f = bg.astype(np.float32)
        out_rgb = bg_f*(1.0 - alpha) + geom_rgb

        if camera_scale != 1.0:
            h, w, _ = out_rgb.shape
            new_w = max(1, int(w * camera_scale))
            new_h = max(1, int(h * camera_scale))
            frame_img = Image.fromarray(out_rgb.astype(np.uint8))
            zoom_img = frame_img.resize((new_w, new_h), resample=Image.BICUBIC)
            x = (new_w - w) // 2
            y = (new_h - h) // 2
            crop = zoom_img.crop((x, y, x + w, y + h))
            out_rgb = np.array(crop, dtype=np.float32)

        return out_rgb.astype(np.uint8)


Cell 5 - LightCraft Render + basic_analyze_wav

In [12]:
class LightCraftRenderer:
    def __init__(self, width: int, height: int, fps: int, svg_paths: List[str]):
        self.width = width
        self.height = height
        self.fps = fps
        self.svg_paths = svg_paths
        self.visual_gen = SVGVisualGenerator(width, height, svg_paths)
        self.audio_data = np.array([])
        self.sample_rate = 0
        self.duration_s = 0.0

    def _load_audio(self, path: str) -> bool:
        if not os.path.exists(path):
            print(f"[Loader] Audio not found: {path}")
            print(f"          Using {MOCK_DURATION_S}s synthetic noise.")
            self.sample_rate = 44100
            self.duration_s = MOCK_DURATION_S
            n = int(self.sample_rate * self.duration_s)
            self.audio_data = np.random.uniform(-0.8, 0.8, n).astype(np.float32)
            return False

        if not HAS_LIBROSA:
            print("[Loader] Librosa not installed. Using synthetic noise.")
            self.sample_rate = 44100
            self.duration_s = MOCK_DURATION_S
            n = int(self.sample_rate * self.duration_s)
            self.audio_data = np.random.uniform(-0.8, 0.8, n).astype(np.float32)
            return False

        print(f"[Loader] Loading audio via Librosa: {path}")
        try:
            y, sr = librosa.load(path, sr=44100)
            self.audio_data = y
            self.sample_rate = sr
            self.duration_s = librosa.get_duration(y=y, sr=sr)
            print(f"[Loader] Duration: {self.duration_s:.2f}s")
            return True
        except Exception as e:
            print(f"[Loader] Librosa failed: {e}")
            print(f"          Using {MOCK_DURATION_S}s synthetic noise.")
            self.sample_rate = 44100
            self.duration_s = MOCK_DURATION_S
            n = int(self.sample_rate * self.duration_s)
            self.audio_data = np.random.uniform(-0.8, 0.8, n).astype(np.float32)
            return False

    def render(self, audio_path: str, out_path: str):
        if not HAS_MOVIEPY:
            print("ERROR: MoviePy not installed. pip install moviepy")
            return

        self._load_audio(audio_path)
        analyzer = SacredAnalyzer(self.audio_data, self.sample_rate, self.duration_s)

        def make_frame(t: float) -> np.ndarray:
            params = analyzer.get_params(t)
            frame_idx = int(t * self.fps)
            if frame_idx % (self.fps * 5) == 0:
                zidx = int(params["astro_phase"]) % ASTRO_CYCLES
                zsign = ZODIAC_ARCHETYPES[zidx]["sign"]
                print(
                    f"[Frame] t={t:6.2f}s | layers={params['layers']} "
                    f"| purity={params['consonance']:.2f} | zodiac={zsign}"
                )
            return self.visual_gen.make_frame(t, params)

        clip = VideoClip(make_frame, duration=self.duration_s).set_fps(self.fps)
        start_t = time.time()

        try:
            audio_clip = AudioFileClip(audio_path)
            final_clip = clip.set_audio(audio_clip)
            print("[Encoder] Rendering LightCraft (with audio)...")
            final_clip.write_videofile(
                out_path,
                fps=self.fps,
                codec="libx264",
                temp_audiofile="temp-audio.m4a",
                remove_temp=True,
                verbose=False,
                logger=None,
            )
            print(f"[Done] Saved → {out_path} ({time.time() - start_t:.2f}s)")
        except Exception as e:
            print(f"[Encoder] Audio attach failed: {e}")
            fallback = out_path.replace(".mp4", "_visual_only.mp4")
            print("[Encoder] Rendering visual-only fallback...")
            clip.write_videofile(
                fallback,
                fps=self.fps,
                codec="libx264",
                verbose=False,
                logger=None,
            )
            print(f"[Done] Saved → {fallback} ({time.time() - start_t:.2f}s)")

def basic_analyze_wav(path: str) -> Dict[str, Any]:
    if not path.lower().endswith(".wav"):
        raise ValueError("Basic mode expects a .wav file.")
    try:
        with wave.open(path, "rb") as wf:
            n_channels = wf.getnchannels()
            sw = wf.getsampwidth()
            fr = wf.getframerate()
            n_frames = wf.getnframes()
            frames = wf.readframes(n_frames)
        if sw != 2:
            raise ValueError("Basic mode only supports 16-bit WAV.")
        fmt = f"{n_frames * n_channels}h"
        audio = np.array(struct.unpack(fmt, frames), dtype=np.float32)
        max16 = 32767.0
        rms = float(np.sqrt(np.mean(audio**2)))
        vol = rms / max16
        zc = float(np.sum(np.abs(np.diff(np.sign(audio)))) / 2.0)
        busy = zc / float(n_frames)
        dur = n_frames / float(fr)
        return {"volume": vol, "busy": busy, "duration": dur}
    except Exception as e:
        print(f"[Basic] WAV analysis error: {e}")
        return {"volume": 0.0, "busy": 0.0, "duration": 10.0}


Cell 6 - Notebook UI: Upload + Resolution + Mode + Render Button

In [13]:
# Widgets
audio_uploader = widgets.FileUpload(
    accept='.wav,.mp3,.m4a,.flac',
    multiple=False,
    description='Upload Audio'
)

mode_dropdown = widgets.Dropdown(
    options=[
        ('Full LightCraft (Librosa, audio-reactive)', 'full'),
        ('Basic WAV Visualizer (simpler, WAV only)', 'basic')
    ],
    value='full',
    description='Mode:',
)

res_dropdown = widgets.Dropdown(
    options=[(f"{w}x{h} {desc}", key) for key, (w,h,desc) in RESOLUTIONS.items()],
    value=1,
    description='Resolution:'
)

out_name = widgets.Text(
    value='lightcraft_output.mp4',
    description='Output:',
    layout=widgets.Layout(width='50%')
)

run_button = widgets.Button(
    description='Render LightCraft',
    button_style='success'
)

log_output = widgets.Output()

controls = widgets.VBox([
    widgets.HTML("<h3>LightCraft Notebook Workstation</h3>"),
    widgets.HTML("<b>1.</b> Upload audio file"),
    audio_uploader,
    widgets.HTML("<b>2.</b> Choose mode & resolution"),
    mode_dropdown,
    res_dropdown,
    widgets.HTML("<b>3.</b> Output filename"),
    out_name,
    run_button,
    log_output
])

display(controls)

def handle_render_clicked(b):
    with log_output:
        clear_output()
        print("[UI] Starting LightCraft render...")

        # Check geometry dir
        try:
            svg_paths = _ensure_geometry_dir(GEOMETRY_DIR)
        except Exception as e:
            print(f"[Error] {e}")
            return

        # Check audio upload
        if len(audio_uploader.value) == 0:
            print("[Error] Please upload an audio file first.")
            return

        # Save uploaded audio to a temp file
        upload_info = list(audio_uploader.value.values())[0]
        audio_bytes = upload_info['content']
        original_name = upload_info['metadata']['name']
        temp_audio_path = os.path.join(os.getcwd(), f"uploaded_{original_name}")

        with open(temp_audio_path, 'wb') as f:
            f.write(audio_bytes)

        print(f"[UI] Saved uploaded audio to: {temp_audio_path}")

        # Resolution
        res_key = res_dropdown.value
        width, height, _ = RESOLUTIONS[res_key]

        # Output path
        output_path = out_name.value.strip()
        if not output_path:
            output_path = "lightcraft_output.mp4"
        if not output_path.lower().endswith(".mp4"):
            output_path += ".mp4"

        mode = mode_dropdown.value

        if mode == 'full':
            print("[UI] Running Full LightCraft mode...")
            renderer = LightCraftRenderer(width, height, RENDER_FPS, svg_paths)
            renderer.render(temp_audio_path, output_path)
        else:
            print("[UI] Running Basic WAV Visualizer mode...")
            if not temp_audio_path.lower().endswith(".wav"):
                print("[Error] Basic mode currently only supports .wav files.")
                return

            stats = basic_analyze_wav(temp_audio_path)
            duration = max(5.0, stats["duration"])
            vg = SVGVisualGenerator(width, height, svg_paths)

            def make_frame(t: float) -> np.ndarray:
                time_ratio = t / duration
                astro_phase = (time_ratio * ASTRO_CYCLES) % ASTRO_CYCLES
                layers = 1 + int(time_ratio * SACRED_NUMBER)
                geom_intensity = min(1.0, stats["volume"] * 2.5)
                pulse = abs(math.sin(t * 2.0))
                params = {
                    "time_s": t,
                    "layers": layers,
                    "geom_intensity": geom_intensity,
                    "consonance": 0.8,
                    "dissonance": 0.2,
                    "pulse": pulse,
                    "astro_phase": astro_phase,
                }
                return vg.make_frame(t, params)

            if not HAS_MOVIEPY:
                print("ERROR: MoviePy not installed. pip install moviepy")
                return

            clip = VideoClip(make_frame, duration=duration).set_fps(RENDER_FPS)
            print("[Basic] Rendering hypnotic LightCraft (no audio sync)...")
            clip.write_videofile(
                output_path,
                fps=RENDER_FPS,
                codec="libx264",
                verbose=False,
                logger=None,
            )
            print(f"[Basic] Saved → {output_path}")

        print("[UI] Done.")

run_button.on_click(handle_render_clicked)
